In [10]:
import pandas as pd
from pathlib import Path

csv_path = Path('/home/ec2-user/LLMs-Scalable-Deliberation/annotation/summary-rating/data_files/test/sum_humanstudy_triplet_full_ring.csv')
df = pd.read_csv(csv_path)


In [11]:
# Count distribution per unique question for question rows
question_df = df[df['type'] == 'question']
counts = question_df['question'].value_counts().rename_axis('question').reset_index(name='count')
counts


,question,count
0,Do you support requiring real-name registratio...,225
1,Do you support the government accepting more r...,225
2,Do you support the government having the autho...,225
3,Do you support the government provide basic he...,225
4,Do you think the current tariff policy under t...,225
5,How has AI changed your life? Please answer br...,225
6,What are your thoughts on Trump’s decision to ...,225
7,What is your opinion on internet influencers (...,225
8,What is your opinion on the rapid update cycle...,225
9,"What is your opinion on tipping, and if given ...",225


In [12]:
# Distribution of summary_a_id under each question
from pathlib import Path

df_a = df.copy()
df_a['summary_a_id'] = df_a['summary_a_id'].astype(str)
mask = df_a['summary_a_id'].notna() & (df_a['summary_a_id'].str.len() > 0) & (df_a['summary_a_id'] != 'nan')

question_summary_a_counts = (
    df_a[mask]
      .groupby(['question', 'summary_a_id'])
      .size()
      .reset_index(name='count')
      .sort_values(['question', 'count'], ascending=[True, False])
)

question_summary_a_counts


,question,summary_a_id,count
0,Do you support requiring real-name registratio...,073b0711079eb28a,3
1,Do you support requiring real-name registratio...,0a208ca9e7fdf00f,3
2,Do you support requiring real-name registratio...,13b031a9de5f4204,3
3,Do you support requiring real-name registratio...,14a059d94a5c384c,3
4,Do you support requiring real-name registratio...,1ce9c77aeaab4f45,3
...,...,...,...
745,"What is your opinion on tipping, and if given ...",f1328cba745e671f,3
746,"What is your opinion on tipping, and if given ...",f5648f8c9a46b2a7,3
747,"What is your opinion on tipping, and if given ...",f5671e09e4b1992d,3
748,"What is your opinion on tipping, and if given ...",f7233d5e7064fecb,3


In [13]:
# Distribution of summary_a_id under each question
from pathlib import Path

df_a = df.copy()
df_a['summary_b_id'] = df_a['summary_b_id'].astype(str)
mask = df_a['summary_b_id'].notna() & (df_a['summary_b_id'].str.len() > 0) & (df_a['summary_b_id'] != 'nan')

question_summary_a_counts = (
    df_a[mask]
      .groupby(['question', 'summary_b_id'])
      .size()
      .reset_index(name='count')
      .sort_values(['question', 'count'], ascending=[True, False])
)

question_summary_a_counts


,question,summary_b_id,count
0,Do you support requiring real-name registratio...,073b0711079eb28a,3
1,Do you support requiring real-name registratio...,0a208ca9e7fdf00f,3
2,Do you support requiring real-name registratio...,13b031a9de5f4204,3
3,Do you support requiring real-name registratio...,14a059d94a5c384c,3
4,Do you support requiring real-name registratio...,1ce9c77aeaab4f45,3
...,...,...,...
745,"What is your opinion on tipping, and if given ...",f1328cba745e671f,3
746,"What is your opinion on tipping, and if given ...",f5648f8c9a46b2a7,3
747,"What is your opinion on tipping, and if given ...",f5671e09e4b1992d,3
748,"What is your opinion on tipping, and if given ...",f7233d5e7064fecb,3


In [14]:
# 检查A-B对是否都有对应的B-A对
print("检查配对对称性...")

# 获取comparison类型的数据
comparison_df = df[df['type'] == 'comparison'].copy()

# 提取所有A-B对
ab_pairs = set()
for _, row in comparison_df.iterrows():
    if pd.notna(row['summary_a_id']) and pd.notna(row['summary_b_id']):
        ab_pairs.add((row['summary_a_id'], row['summary_b_id']))

print(f"总共有 {len(ab_pairs)} 个A-B对")

# 检查每个A-B对是否有对应的B-A对
missing_ba_pairs = []
existing_ba_pairs = []

for a_id, b_id in ab_pairs:
    ba_pair = (b_id, a_id)
    if ba_pair in ab_pairs:
        existing_ba_pairs.append((a_id, b_id))
    else:
        missing_ba_pairs.append((a_id, b_id))

print(f"\n对称性检查结果:")
print(f"- 有对应B-A对的A-B对: {len(existing_ba_pairs)}")
print(f"- 缺少对应B-A对的A-B对: {len(missing_ba_pairs)}")
print(f"- 对称率: {len(existing_ba_pairs) / len(ab_pairs) * 100:.2f}%")

if missing_ba_pairs:
    print(f"\n前10个缺少B-A对的A-B对:")
    for i, (a_id, b_id) in enumerate(missing_ba_pairs[:10]):
        print(f"  {i+1}. A={a_id} -> B={b_id} (缺少 B={b_id} -> A={a_id})")
else:
    print("\n✅ 所有A-B对都有对应的B-A对！")


检查配对对称性...
总共有 2250 个A-B对

对称性检查结果:
- 有对应B-A对的A-B对: 0
- 缺少对应B-A对的A-B对: 2250
- 对称率: 0.00%

前10个缺少B-A对的A-B对:
  1. A=fded4eacfb56fbd4 -> B=759519fd091c850e (缺少 B=759519fd091c850e -> A=fded4eacfb56fbd4)
  2. A=a368f52fd450713b -> B=bcf20dbee5035406 (缺少 B=bcf20dbee5035406 -> A=a368f52fd450713b)
  3. A=0d5f7e2634bbaf47 -> B=7955fefd70ef263e (缺少 B=7955fefd70ef263e -> A=0d5f7e2634bbaf47)
  4. A=3c59c2c1b5ba05bc -> B=f30703e0be603dc1 (缺少 B=f30703e0be603dc1 -> A=3c59c2c1b5ba05bc)
  5. A=208b9ea8a7052463 -> B=d1742101510cc8bc (缺少 B=d1742101510cc8bc -> A=208b9ea8a7052463)
  6. A=46896e968fb5a829 -> B=fecd823b5bcb5092 (缺少 B=fecd823b5bcb5092 -> A=46896e968fb5a829)
  7. A=bf45ab5996d45392 -> B=32dcfde4cad2ea2f (缺少 B=32dcfde4cad2ea2f -> A=bf45ab5996d45392)
  8. A=77d639cfdef1794a -> B=ae10460ea5c8b5e6 (缺少 B=ae10460ea5c8b5e6 -> A=77d639cfdef1794a)
  9. A=90c2c44367d8ba8c -> B=6bc9faf246886197 (缺少 B=6bc9faf246886197 -> A=90c2c44367d8ba8c)
  10. A=586806a26aa4cbb2 -> B=121889143adb3c00 (缺少 B=121889143adb

In [15]:
# 更详细的对称性分析
print("详细分析配对结构...")

# 按问题分组分析对称性
question_symmetry = {}
for question in comparison_df['question'].unique():
    question_data = comparison_df[comparison_df['question'] == question]
    
    # 提取该问题下的所有A-B对
    question_pairs = set()
    for _, row in question_data.iterrows():
        if pd.notna(row['summary_a_id']) and pd.notna(row['summary_b_id']):
            question_pairs.add((row['summary_a_id'], row['summary_b_id']))
    
    # 检查对称性
    symmetric_pairs = 0
    asymmetric_pairs = 0
    
    for a_id, b_id in question_pairs:
        ba_pair = (b_id, a_id)
        if ba_pair in question_pairs:
            symmetric_pairs += 1
        else:
            asymmetric_pairs += 1
    
    question_symmetry[question] = {
        'total_pairs': len(question_pairs),
        'symmetric_pairs': symmetric_pairs,
        'asymmetric_pairs': asymmetric_pairs,
        'symmetry_rate': symmetric_pairs / len(question_pairs) * 100 if question_pairs else 0
    }

# 显示每个问题的对称性统计
print(f"\n各问题的对称性分析:")
for i, (question, stats) in enumerate(question_symmetry.items()):
    print(f"\n问题 {i+1}: {question[:50]}...")
    print(f"  总配对数: {stats['total_pairs']}")
    print(f"  对称配对: {stats['symmetric_pairs']}")
    print(f"  非对称配对: {stats['asymmetric_pairs']}")
    print(f"  对称率: {stats['symmetry_rate']:.1f}%")

# 总体统计
total_pairs = sum(stats['total_pairs'] for stats in question_symmetry.values())
total_symmetric = sum(stats['symmetric_pairs'] for stats in question_symmetry.values())
total_asymmetric = sum(stats['asymmetric_pairs'] for stats in question_symmetry.values())

print(f"\n总体对称性统计:")
print(f"- 总配对数: {total_pairs}")
print(f"- 对称配对数: {total_symmetric}")
print(f"- 非对称配对数: {total_asymmetric}")
print(f"- 整体对称率: {total_symmetric / total_pairs * 100:.2f}%")


详细分析配对结构...

各问题的对称性分析:

问题 1: Do you support requiring real-name registration on...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 2: Do you support the government accepting more refug...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 3: Do you support the government having the authority...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 4: Do you support the government provide basic health...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 5: Do you think the current tariff policy under the T...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 6: How has AI changed your life? Please answer briefl...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 7: What are your thoughts on Trump’s decision to cut ...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 8: What is your opinion on internet influencers (e.g....
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 9: What is your opinion on the rapid update cycle of ...
  总配对数: 225
  对称配对: 0
  非对称配对: 225
  对称率: 0.0%

问题 

In [16]:
# 验证环形算法的特性
print("验证环形算法特性...")

# 统计每个summary作为A和作为B出现的次数
a_counts = {}
b_counts = {}

for _, row in comparison_df.iterrows():
    if pd.notna(row['summary_a_id']) and pd.notna(row['summary_b_id']):
        a_id = row['summary_a_id']
        b_id = row['summary_b_id']
        
        a_counts[a_id] = a_counts.get(a_id, 0) + 1
        b_counts[b_id] = b_counts.get(b_id, 0) + 1

print(f"\n作为A出现的统计:")
print(f"- 不同的A数量: {len(a_counts)}")
print(f"- A出现次数分布: {sorted(set(a_counts.values()))}")
print(f"- 每个A的平均出现次数: {sum(a_counts.values()) / len(a_counts):.1f}")

print(f"\n作为B出现的统计:")
print(f"- 不同的B数量: {len(b_counts)}")
print(f"- B出现次数分布: {sorted(set(b_counts.values()))}")
print(f"- 每个B的平均出现次数: {sum(b_counts.values()) / len(b_counts):.1f}")

# 检查是否每个summary都出现了相同次数
a_count_values = list(a_counts.values())
b_count_values = list(b_counts.values())

print(f"\n一致性检查:")
print(f"- 所有A出现次数是否相同: {len(set(a_count_values)) == 1}")
print(f"- 所有B出现次数是否相同: {len(set(b_count_values)) == 1}")

if len(set(a_count_values)) == 1:
    print(f"- 每个A恰好出现: {a_count_values[0]} 次")
if len(set(b_count_values)) == 1:
    print(f"- 每个B恰好出现: {b_count_values[0]} 次")

# 检查A和B的集合是否相同
a_set = set(a_counts.keys())
b_set = set(b_counts.keys())
print(f"- A和B的集合是否相同: {a_set == b_set}")
print(f"- A集合大小: {len(a_set)}, B集合大小: {len(b_set)}")

if a_set != b_set:
    only_a = a_set - b_set
    only_b = b_set - a_set
    print(f"- 只作为A出现: {len(only_a)} 个")
    print(f"- 只作为B出现: {len(only_b)} 个")


验证环形算法特性...

作为A出现的统计:
- 不同的A数量: 750
- A出现次数分布: [3]
- 每个A的平均出现次数: 3.0

作为B出现的统计:
- 不同的B数量: 750
- B出现次数分布: [3]
- 每个B的平均出现次数: 3.0

一致性检查:
- 所有A出现次数是否相同: True
- 所有B出现次数是否相同: True
- 每个A恰好出现: 3 次
- 每个B恰好出现: 3 次
- A和B的集合是否相同: True
- A集合大小: 750, B集合大小: 750


In [ ]:
# 分析标注结果数据
print("="*60)
print("分析标注结果数据 (annotated_instances.csv)")
print("="*60)

# 加载标注结果数据
annotation_path = '/home/ec2-user/LLMs-Scalable-Deliberation/annotation/summary-rating/annotation_output/full/annotated_instances.csv'
annotation_df = pd.read_csv(annotation_path)

print(f"标注数据基本信息:")
print(f"- 总行数: {len(annotation_df)}")
print(f"- 总列数: {len(annotation_df.columns)}")
print(f"- 不同用户数: {annotation_df['user'].nunique()}")
print(f"- 不同实例数: {annotation_df['instance_id'].nunique()}")

# 显示列名
print(f"\n列名列表:")
for i, col in enumerate(annotation_df.columns, 1):
    print(f"  {i:2d}. {col}")


In [ ]:
# 分析用户标注行为
print("用户标注行为分析:")

# 每个用户的标注数量
user_counts = annotation_df['user'].value_counts()
print(f"\n用户标注数量统计:")
print(f"- 最多标注: {user_counts.max()} 个实例")
print(f"- 最少标注: {user_counts.min()} 个实例") 
print(f"- 平均标注: {user_counts.mean():.1f} 个实例")
print(f"- 中位数标注: {user_counts.median():.1f} 个实例")

print(f"\n用户标注数量分布:")
user_count_dist = user_counts.value_counts().sort_index()
for count, num_users in user_count_dist.items():
    print(f"  标注 {count:2d} 个实例的用户: {num_users:2d} 人")

# 显示前10个最活跃用户
print(f"\n前10个最活跃用户:")
for i, (user_id, count) in enumerate(user_counts.head(10).items(), 1):
    print(f"  {i:2d}. {user_id}: {count} 个标注")


In [ ]:
# 分析标注问题类型和分布
print("标注问题类型分析:")

# 提取实例类型 (从instance_id推断)
annotation_df['instance_type'] = annotation_df['instance_id'].apply(
    lambda x: 'triplet_comparison' if 'comparison' in x 
    else 'triplet_rating' if 'rating' in x 
    else 'triplet_question' if 'question' in x
    else 'unknown'
)

instance_type_counts = annotation_df['instance_type'].value_counts()
print(f"\n实例类型分布:")
for inst_type, count in instance_type_counts.items():
    print(f"  {inst_type}: {count} 个标注")

# 分析comparison类型的标注
comparison_df = annotation_df[annotation_df['instance_type'] == 'triplet_comparison'].copy()
if len(comparison_df) > 0:
    print(f"\n配对比较标注分析 ({len(comparison_df)} 个标注):")
    
    # 分析配对比较问题的回答
    comparison_cols = [col for col in annotation_df.columns if 'Which summary' in col]
    print(f"- 配对比较问题数量: {len(comparison_cols)}")
    
    for col in comparison_cols[:4]:  # 显示前4个问题
        if col in comparison_df.columns:
            response_counts = comparison_df[col].value_counts()
            print(f"\n  问题: {col.split(':::')[0]}")
            print(f"    回答分布:")
            for response, count in response_counts.items():
                if pd.notna(response):
                    print(f"      {response}: {count} 次")

# 分析rating类型的标注  
rating_df = annotation_df[annotation_df['instance_type'] == 'triplet_rating'].copy()
if len(rating_df) > 0:
    print(f"\n评分标注分析 ({len(rating_df)} 个标注):")
    
    # 分析评分问题
    rating_cols = [col for col in annotation_df.columns if 'To what extent' in col or 'How informative' in col]
    print(f"- 评分问题数量: {len(rating_cols)}")
    
    for col in rating_cols[:3]:  # 显示前3个问题
        if col in rating_df.columns:
            response_counts = rating_df[col].value_counts()
            print(f"\n  问题: {col.split(':::')[0]}")
            print(f"    评分分布:")
            for response, count in response_counts.items():
                if pd.notna(response):
                    print(f"      {response}: {count} 次")


In [ ]:
# 分析标注完成度和数据质量
print("标注完成度和数据质量分析:")

# 检查缺失值
print(f"\n缺失值统计:")
missing_counts = annotation_df.isnull().sum()
total_cells = len(annotation_df)

print(f"- 总行数: {total_cells}")
for col in annotation_df.columns:
    missing_count = missing_counts[col]
    missing_pct = (missing_count / total_cells) * 100
    if missing_count > 0:
        print(f"  {col}: {missing_count} 个缺失 ({missing_pct:.1f}%)")

# 分析文本回答
text_answer_col = 'answer:::text_box'
if text_answer_col in annotation_df.columns:
    text_answers = annotation_df[text_answer_col].dropna()
    print(f"\n文本回答分析:")
    print(f"- 有文本回答的记录: {len(text_answers)} 个")
    print(f"- 文本回答完成率: {len(text_answers) / len(annotation_df) * 100:.1f}%")
    
    if len(text_answers) > 0:
        # 文本长度分布
        text_lengths = text_answers.str.len()
        print(f"- 平均文本长度: {text_lengths.mean():.1f} 字符")
        print(f"- 最短文本: {text_lengths.min()} 字符")
        print(f"- 最长文本: {text_lengths.max()} 字符")
        
        # 显示几个示例文本回答
        print(f"\n文本回答示例 (前3个):")
        for i, answer in enumerate(text_answers.head(3), 1):
            print(f"  {i}. {answer[:100]}{'...' if len(answer) > 100 else ''}")

# 检查标注一致性
print(f"\n标注一致性检查:")
scale_cols = [col for col in annotation_df.columns if ':::scale_' in col]
print(f"- 量表问题数量: {len(scale_cols)}")

if scale_cols:
    # 检查量表回答的分布
    print(f"\n量表回答分布:")
    for col in scale_cols[:5]:  # 检查前5个量表问题
        if col in annotation_df.columns:
            responses = annotation_df[col].dropna()
            if len(responses) > 0:
                unique_responses = sorted(responses.unique())
                print(f"  {col.split(':::')[0][:40]}...")
                print(f"    回答选项: {unique_responses}")
                print(f"    回答数量: {len(responses)}")


In [ ]:
# 高级分析：用户偏好和模式
print("用户偏好和标注模式分析:")

# 分析配对比较的偏好模式
comparison_preference_cols = [col for col in annotation_df.columns if 'Which summary' in col and 'scale_' in col]

if comparison_preference_cols:
    print(f"\n配对比较偏好分析:")
    
    for col in comparison_preference_cols:
        if col in annotation_df.columns:
            preferences = annotation_df[col].dropna()
            if len(preferences) > 0:
                question_type = col.split(':::')[0]
                scale_type = col.split(':::')[1]
                
                print(f"\n  {question_type} ({scale_type}):")
                pref_counts = preferences.value_counts()
                total_responses = len(preferences)
                
                for pref, count in pref_counts.items():
                    pct = (count / total_responses) * 100
                    print(f"    {pref}: {count} 次 ({pct:.1f}%)")

# 分析评分模式
rating_cols = [col for col in annotation_df.columns if ('To what extent' in col or 'How informative' in col or 'neutral and balanced' in col or 'approve' in col) and 'scale_' in col]

if rating_cols:
    print(f"\n评分模式分析:")
    
    # 按评分类型分组
    rating_questions = {}
    for col in rating_cols:
        question = col.split(':::')[0]
        scale = col.split(':::')[1]
        
        if question not in rating_questions:
            rating_questions[question] = []
        rating_questions[question].append(col)
    
    for question, cols in list(rating_questions.items())[:3]:  # 分析前3个问题类型
        print(f"\n  {question[:50]}...")
        
        # 合并所有量表的回答
        all_responses = []
        for col in cols:
            responses = annotation_df[col].dropna()
            all_responses.extend(responses.tolist())
        
        if all_responses:
            response_counts = pd.Series(all_responses).value_counts()
            total = len(all_responses)
            
            print(f"    总回答数: {total}")
            for response, count in response_counts.items():
                pct = (count / total) * 100
                print(f"    {response}: {count} 次 ({pct:.1f}%)")

# 分析用户一致性
print(f"\n用户标注一致性分析:")
if len(annotation_df) > 0:
    # 计算每个用户的标注分散度
    user_consistency = {}
    
    for user in annotation_df['user'].unique():
        user_data = annotation_df[annotation_df['user'] == user]
        
        # 分析该用户在量表问题上的回答分散度
        user_responses = []
        for col in scale_cols[:10]:  # 检查前10个量表问题
            if col in user_data.columns:
                responses = user_data[col].dropna()
                user_responses.extend(responses.tolist())
        
        if user_responses:
            # 计算标准差作为一致性指标
            try:
                numeric_responses = [float(r) for r in user_responses if str(r).replace('.', '').isdigit()]
                if numeric_responses:
                    consistency = pd.Series(numeric_responses).std()
                    user_consistency[user] = consistency
            except:
                pass
    
    if user_consistency:
        consistency_series = pd.Series(user_consistency)
        print(f"- 用户数量: {len(user_consistency)}")
        print(f"- 平均标注一致性 (标准差): {consistency_series.mean():.2f}")
        print(f"- 最一致用户标准差: {consistency_series.min():.2f}")
        print(f"- 最不一致用户标准差: {consistency_series.max():.2f}")
        
        # 显示最一致和最不一致的用户
        most_consistent = consistency_series.idxmin()
        least_consistent = consistency_series.idxmax()
        print(f"- 最一致用户: {most_consistent} (std: {consistency_series[most_consistent]:.2f})")
        print(f"- 最不一致用户: {least_consistent} (std: {consistency_series[least_consistent]:.2f})")
